In [1]:
import pandas as pd
import numpy as np

import sidetable as stb

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew
from scipy.stats import kurtosistest
from scipy import stats

import pickle
import listas as lst

from datetime import datetime

In [2]:
df = pd.read_pickle("../data/bikes_predicciones_reg.pkl")
df.sample()

,estacion,año,mes,festivo,dia_sem,laborable,prevision_tiempo,sensacion_termica,humedad,velocidad_viento
698,Otoño,2019,noviembre,0,sabado,1,1,16.19335,64.9583,3.917544


In [3]:
with open ("../data/robust_registrado.pkl", "rb") as file:
    robust_registrado = pickle.load(file) 

In [4]:
with open ("../data/dt_registrado_pred.pkl", "rb") as modelo_registrado:
    dt_registrado_pred = pickle.load(modelo_registrado)

In [5]:
# creamos un diccionario con datos ficticios:

dicc_usuario_reg = {'estacion': 'Verano', 'año': '2019', 'mes': 'septiembre', 'festivo': 0, 'dia_sem': 'domingo', 'laborable': 0,
                'prevision_tiempo': 1, 'sensacion_termica': 30.52440, 'humedad': 64.6667, 'velocidad_viento': 19.000061}

In [6]:
# creamos df

df_usuario_reg = pd.DataFrame(dicc_usuario_reg, index= [0])


In [7]:
df_usuario_reg

,estacion,año,mes,festivo,dia_sem,laborable,prevision_tiempo,sensacion_termica,humedad,velocidad_viento
0,Verano,2019,septiembre,0,domingo,0,1,30.5244,64.6667,19.000061


In [8]:
df_cat_reg = df_usuario_reg.select_dtypes(exclude= np.number)

In [9]:
lst.lista_dict_reg # viene del archivo listas.py

[{'Invierno': 0, 'Otoño': 2, 'Primavera': 2.4, 'Verano': 2.2},
 {'2018': 0, '2019': 2},
 {'enero': 0,
  'febrero': 0.5,
  'marzo': 1,
  'diciembre': 1.5,
  'abril': 2,
  'noviembre': 2,
  'mayo': 2.2,
  'octubre': 2.2,
  'julio': 2.2,
  'septiembre': 2.5,
  'agosto': 2.5,
  'junio': 2.7},
 {0: 1, 1: 0},
 {'miercoles': 1,
  'jueves': 1.3,
  'viernes': 1.3,
  'sabado': 1,
  'martes': 0,
  'lunes': 0,
  'domingo': 1},
 {0: 0, 1: 1},
 {3: 0, 2: 2, 1: 2.5}]

In [10]:
df = pd.DataFrame()

for dic, col in zip(lst.lista_dict_reg, df_cat_reg.columns):   
    
    df[f'{col}_map'] = df_cat_reg[col].map(dic)
    
df.head()

,estacion_map,año_map,mes_map,dia_sem_map
0,2.2,2,2.5,NaN


In [14]:
df_usuario_reg = df_usuario_reg.reindex(columns=['sensacion_termica', 'humedad', 'velocidad_viento', 'estacion_map',
       'año_map', 'mes_map',  'festivo_map','dia_sem_map', 'laborable_map',
       'prevision_tiempo_map'])

In [15]:
df_num_reg = df_usuario_reg[["sensacion_termica", "humedad", "velocidad_viento"]]

In [16]:
df_num_reg

,sensacion_termica,humedad,velocidad_viento
0,30.5244,64.6667,19.000061


In [17]:
robust_registrado.transform(df_num_reg)

array([[0.45412707, 0.09651821, 1.04416759]])

In [18]:
dt_registrado_pred.predict(df_usuario_reg)

array([5819.7])

Como podemos comprobar, el modelo predice 5819 bicicletas alquiladas mientras que en los datos originales el total de bicicletas de miembros registrados suman 5883. La diferencia es mínima, apenas 64 bicis. 

In [19]:
df = pd.read_pickle("../data/outliers_gestion.pkl")
df[df["sensacion_termica"] == 30.52440]

,fecha,estacion,año,mes,festivo,dia_sem,laborable,prevision_tiempo,sensacion_termica,humedad,velocidad_viento,alquiler_casual,alquiler_registrado,total_alquileres
629,2019-09-22,Verano,2019,septiembre,0,domingo,0,1,30.5244,64.6667,19.000061,2512,5883,8395
